## Постановка задачи
Загрузим данные, приведем их к числовым, заполним пропуски, нормализуем данные и оптимизируем память.

Разделим выборку на обучающую/проверочную в соотношении 80/20.

Построим ансамбль решающих деревьев, используя патентованный градиентный бустинг Яндекса (CatBoost). Используем перекрестную проверку, чтобы найти наилучшие параметры ансамбля.

Проведем предсказание и проверим качество через каппа-метрику.

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from catboost import Pool, CatBoostClassifier
from sklearn import preprocessing

### Загрузка данных

In [2]:
data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


### Предобработка данных

In [3]:
data["Product_Info_2_1"] = data["Product_Info_2"].str.slice(0, 1)
data["Product_Info_2_2"] = pd.to_numeric(data["Product_Info_2"].str.slice(1, 2))
data.drop(labels=["Product_Info_2"], axis=1, inplace=True)
for l in data["Product_Info_2_1"].unique():
    data["Product_Info_2_1" + l] = data["Product_Info_2_1"].isin([l]).astype("int8")
data.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)
data.fillna(value=-1, inplace=True)
data["Response"] = data["Response"] - 1

### Набор столбцов для расчета

In [4]:
columns_groups = ["Insurance_History", "InsurеdInfo", "Medical_Keyword",
                  "Family_Hist", "Medical_History", "Product_Info"]
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print (columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

### Нормализация данных

In [5]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data,
                                                     columns=columns)))
columns_transformed = data_transformed.columns
data_transformed["Response"] = data["Response"]

### Оптимизация памяти

In [6]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [7]:
data_transformed = reduce_mem_usage(data_transformed)
print (data_transformed.info())

Потребление памяти меньше на 40.49 Мб (минус 75.1 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 119 entries, 0 to Response
dtypes: float16(118), int8(1)
memory usage: 13.4 MB
None


### Разделение данных
Преобразуем выборки в отдельные наборы данных

In [8]:
data_train, data_test = train_test_split(data_transformed,
                                         test_size=0.2)
data_train = pd.DataFrame(data_train)
data_test = pd.DataFrame(data_test)
print (data_train.head())

              0         1         2         3         4         5         6  \
29740  0.426270  1.003906  1.424805 -0.081970  0.611816 -0.169434  0.862305   
53953  0.543945  1.003906  0.667969  0.049133  0.611816 -0.169434  0.862305   
41278 -1.618164 -1.200195 -0.694336 -1.456055 -1.634766 -0.169434  0.862305   
26944  0.097351 -0.710449 -0.618652  0.658203  0.611816 -0.169434 -1.159180   
35805 -0.866211 -0.220581  0.441162 -0.935547  0.611816  5.902344 -1.159180   

              7         8         9  ...       109       110       111  \
29740 -1.013672  0.863281 -0.928711  ... -0.083679  0.441650 -0.149292   
53953 -1.013672  0.863770 -0.928711  ... -0.083679  0.441650 -0.149292   
41278 -1.013672  0.867676 -0.928711  ... -0.083679 -2.263672 -0.149292   
26944  1.100586 -1.156250  1.130859  ... -0.083679  0.441650 -0.149292   
35805  1.100586 -1.156250  1.130859  ... -0.083679  0.441650 -0.149292   

            112       113       114       115       116      117  Response  
297

### CatBoost
Основные преимущества: умение работать с категориальными (номинативными) признаками и бОльшая точность, чем LighGBM

Алгоритм запускается сразу на всех ядрах процессора, это существенно ускоряет работу.

В качестве ансамблирования выберем метод опорных векторов (MVS), он ранее показал хорошую точность (и для CatBoost он тоже повышает точность на рассматриваемых данных).

In [9]:
x = pd.DataFrame(data_train, columns=columns_transformed)
train_dataset = Pool(data=x, label=data_train["Response"])
model = CatBoostClassifier(iterations=10, learning_rate=0.57,
                random_seed=17, depth=6, loss_function="MultiClass",
                bootstrap_type="MVS", custom_metric="WKappa")

Диапазон тестирования параметров модели ограничен только вычислительной мощностью. Для проверки модели имеет смысл провести индивидуальные перекрестные проверки для каждого параметра в отдельности, затем в итоговой проверке перепроверить самые лучшие найденные параметры с отклонением +/-10%.

Гиперпараметры оптимизации:
* depth - максимальная глубина деревьев,
* learning_rate - скорость обучения
* l2_leaf_reg - L2 параметр регуляризации для функции стоимости

In [10]:
cb_params = {
    "depth": range(5,8),
    'learning_rate': np.arange(0.56,0.59,0.01),
    'l2_leaf_reg': range(1,5),
}
cb_grid = model.grid_search(cb_params, cv=5, X=x,
                y=data_train["Response"], verbose=True)
print (cb_grid["params"])

0:	loss: 1.2816254	best: 1.2816254 (0)	total: 4.13s	remaining: 2m 24s
1:	loss: 1.2778343	best: 1.2778343 (1)	total: 7.97s	remaining: 2m 15s
2:	loss: 1.2762517	best: 1.2762517 (2)	total: 11.7s	remaining: 2m 9s
3:	loss: 1.2771619	best: 1.2762517 (2)	total: 15.4s	remaining: 2m 3s
4:	loss: 1.2856658	best: 1.2762517 (2)	total: 19.1s	remaining: 1m 58s
5:	loss: 1.2788533	best: 1.2762517 (2)	total: 22.8s	remaining: 1m 53s
6:	loss: 1.2788416	best: 1.2762517 (2)	total: 26.6s	remaining: 1m 50s
7:	loss: 1.2813785	best: 1.2762517 (2)	total: 30.5s	remaining: 1m 46s
8:	loss: 1.2885144	best: 1.2762517 (2)	total: 34.9s	remaining: 1m 44s
9:	loss: 1.2803815	best: 1.2762517 (2)	total: 38.6s	remaining: 1m 40s
10:	loss: 1.2824530	best: 1.2762517 (2)	total: 42.2s	remaining: 1m 35s
11:	loss: 1.2817148	best: 1.2762517 (2)	total: 46.1s	remaining: 1m 32s
12:	loss: 1.2697193	best: 1.2697193 (12)	total: 51.1s	remaining: 1m 30s
13:	loss: 1.2715684	best: 1.2697193 (12)	total: 56s	remaining: 1m 28s
14:	loss: 1.270972

Выведем самые оптимальные параметры и построим итоговую модель

In [12]:
print (cb_grid["params"])
model = CatBoostClassifier(iterations=100,
        learning_rate=cb_grid["params"]["learning_rate"],
        depth=cb_grid["params"]["depth"],
        l2_leaf_reg=cb_grid["params"]["l2_leaf_reg"],
        random_seed=17, loss_function="MultiClass",
        bootstrap_type="MVS", custom_metric="WKappa")

{'depth': 7, 'l2_leaf_reg': 1, 'learning_rate': 0.5800000000000001}


In [13]:
model.fit(train_dataset)

0:	learn: 1.5260121	total: 1.64s	remaining: 2m 42s
1:	learn: 1.4298948	total: 2.86s	remaining: 2m 19s
2:	learn: 1.3249085	total: 4.11s	remaining: 2m 12s
3:	learn: 1.2901868	total: 5.3s	remaining: 2m 7s
4:	learn: 1.2671400	total: 6.6s	remaining: 2m 5s
5:	learn: 1.2555368	total: 7.69s	remaining: 2m
6:	learn: 1.2450711	total: 8.91s	remaining: 1m 58s
7:	learn: 1.2376520	total: 10s	remaining: 1m 55s
8:	learn: 1.2261509	total: 11.3s	remaining: 1m 53s
9:	learn: 1.2191708	total: 12.5s	remaining: 1m 52s
10:	learn: 1.2109959	total: 13.6s	remaining: 1m 50s
11:	learn: 1.2013338	total: 14.9s	remaining: 1m 49s
12:	learn: 1.1945134	total: 16.2s	remaining: 1m 48s
13:	learn: 1.1889184	total: 17.3s	remaining: 1m 46s
14:	learn: 1.1796621	total: 18.5s	remaining: 1m 44s
15:	learn: 1.1730491	total: 19.7s	remaining: 1m 43s
16:	learn: 1.1690535	total: 20.9s	remaining: 1m 41s
17:	learn: 1.1637374	total: 22s	remaining: 1m 40s
18:	learn: 1.1583403	total: 23.2s	remaining: 1m 39s
19:	learn: 1.1521315	total: 24.5s	

### Предсказание данных и оценка модели

In [14]:
x_test = pd.DataFrame(data_test, columns=columns_transformed)
data_test["target"] = model.predict(Pool(data=x_test))

Кластеризация дает 0.192, kNN(100) - 0.3, лог. регрессия - 0.512/0.496, SVM - 0.95, реш. дерево - 0.3, случайный лес - 0.487, XGBoost - 0.536, градиентный бустинг - 0.56, LightGBM - 0.569

In [15]:
print ("CatBoost:",
      round(cohen_kappa_score(data_test["target"],
            data_test["Response"], weights="quadratic"), 3))

CatBoost: 0.538


### Матрица неточностей

In [16]:
print ("CatBoost\n",
      confusion_matrix(data_test["target"], data_test["Response"]))

CatBoost
 [[ 299  188   19   13   69  120   41   31]
 [ 178  332    7    4  118  110   29   28]
 [  29   24   82   22    2    4    0    0]
 [  38   31   53  198    0    4    0    2]
 [  86  128   11    0  515  111    7    4]
 [ 240  261    8   17  203 1138  288  147]
 [ 142  108    4   10   52  272  702  177]
 [ 268  268    7   33   84  438  576 3497]]
